In [1]:
import sys
sys.path.append("/lakehouse/default/Files/code")
from json_log import save_json_log, check_json_log_exists

from pyspark.sql import functions as F
from pyspark.sql import SparkSession
from notebookutils import mssparkutils
import json


StatementMeta(, 466a9824-468e-4c75-b245-7fb02b900cbd, 3, Finished, Available, Finished)

In [2]:
lakehouse_name = "lkh_iemc"  # your target lakehouse
date_partition = "hour"
date_diff_limit = 20
log_table = "lh_bronze.brz_json_log"

StatementMeta(, 466a9824-468e-4c75-b245-7fb02b900cbd, 4, Finished, Available, Finished)

In [3]:
# -----------------------------
# 1. Spark Session & Config
# -----------------------------
config_file = "Files/config_json/region_geo_ids.json"
spark = (
    SparkSession.builder
    .appName("JSON_to_Silver")
    .config("spark.fabric.lakehouse.name", lakehouse_name)
    .getOrCreate()
)

StatementMeta(, 466a9824-468e-4c75-b245-7fb02b900cbd, 5, Finished, Available, Finished)

In [4]:
# Get a list of old files from brz_json_log
#df_old_files = spark.sql(f"SELECT source_file, ingestion_date, DATEDIFF({date_partition}, ingestion_date, NOW()) as D FROM {log_table} WHERE DATEDIFF({date_partition}, ingestion_date, NOW()) >={date_diff_limit}")
#display(df_old_files)


StatementMeta(, 466a9824-468e-4c75-b245-7fb02b900cbd, 6, Finished, Available, Finished)

In [5]:
# Get list of old files
df_old_files = spark.sql(f"""
    SELECT source_file 
    FROM {log_table} 
    WHERE DATEDIFF({date_partition}, ingestion_date, NOW()) >= {date_diff_limit}
""")

# Remove leading "Files/", extract directory path and file name separately
df_files = (
    df_old_files
    # Remove leading "Files/"
    .withColumn("clean_path", F.expr("substring(source_file, 7, length(source_file)-6)"))
    # Extract file name
    .withColumn("file_name", F.regexp_extract("clean_path", r"([^/]+)$", 1))
    # Extract directory only
    .withColumn("file_path", F.regexp_replace("clean_path", r"/[^/]+$", ""))  # remove the file name
)

# Convert to dict: directory path -> file name
file_dict =  [{"file_path": row["file_path"], "file_name": row["file_name"]} for row in df_files.collect()]

#print(file_dict)
mssparkutils.notebook.exit(json.dumps(file_dict))



StatementMeta(, 466a9824-468e-4c75-b245-7fb02b900cbd, 7, Finished, Available, Finished)

ExitValue: [{"file_path": "bronze/REDATA/data/generacion/estructura-generacion-emisiones-asociadas/month", "file_name": "brz-principado-de-asturias-generacion-estructura-generacion-emisiones-asociadas-month-2025-10-01t11-05-25-285679z.json"}, {"file_path": "bronze/REDATA/data/generacion/estructura-generacion-emisiones-asociadas/month", "file_name": "brz-principado-de-asturias-generacion-estructura-generacion-emisiones-asociadas-month-2025-10-02t09-53-31-204331z.json"}, {"file_path": "bronze/REDATA/data/generacion/estructura-generacion-emisiones-asociadas/month", "file_name": "brz-principado-de-asturias-generacion-estructura-generacion-emisiones-asociadas-month-2025-10-02t10-03-07-867898z.json"}, {"file_path": "bronze/REDATA/data/generacion/estructura-generacion-emisiones-asociadas/month", "file_name": "brz-comunidad-valenciana-generacion-estructura-generacion-emisiones-asociadas-month-2025-10-02t09-53-38-070854z.json"}, {"file_path": "bronze/REDATA/data/generacion/estructura-generacion